In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
# Download training data from open datasets
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

In [3]:
# Pass Dataset to DataLoader - wraps as iterable over dataset enabling automaic batching, sampling, shuffling, and multiprocess dataloading

# batch size = 64 -> each element in the dataloader iterable will return a batch of 64 features and labels
batch_size = 64

# Create data loader
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

# look at shape
for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")    # each X batch is 64 x 1 x 28 x 28
    print(f"Shape of y: {y.shape} {y.dtype}")       # each y batch is 64
    break # only print one

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


# Creating Models
- define neural network in pytorch with class that inherits nn.Module
- define layers of network in the `__init` function of class
- specify how data will pass through network in the `foward` function
- use GPU if available to accelerate

In [7]:
# Get cpu or gpu device for training
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using {device} device")

# Define Model
class NeuralNetwork(nn.Module):
    # 28*28 neuron input layer -> 515 neuron hidden layer -> 10 neuron output layer
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


model = NeuralNetwork().to(device)
print(model)


Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


# Training the model

In [8]:
loss_fn = nn.CrossEntropyLoss() # loss function
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

# Each training loop, model makes predictions of training dataset (fed in batches) and backpropagates the prediction error to adjust the model's parameters

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()   # set model into train mode
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Comptue prediction error
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch+1)*len(X)
            print(f'loss{loss:>7f} [{current:>5d}/{size:>5d}]')

# Test model
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():           # No gradient calculation - will not call backward
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

# Train
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss2.291563 [   64/60000]
loss2.289948 [ 6464/60000]
loss2.268766 [12864/60000]
loss2.265725 [19264/60000]
loss2.249155 [25664/60000]
loss2.205088 [32064/60000]
loss2.216104 [38464/60000]
loss2.185799 [44864/60000]
loss2.180720 [51264/60000]
loss2.131809 [57664/60000]
Test Error: 
 Accuracy: 41.4%, Avg loss: 2.142452 

Epoch 2
-------------------------------
loss2.154201 [   64/60000]
loss2.149074 [ 6464/60000]
loss2.090337 [12864/60000]
loss2.102375 [19264/60000]
loss2.054908 [25664/60000]
loss1.991448 [32064/60000]
loss2.004609 [38464/60000]
loss1.945644 [44864/60000]
loss1.934270 [51264/60000]
loss1.840798 [57664/60000]
Test Error: 
 Accuracy: 61.7%, Avg loss: 1.860851 

Epoch 3
-------------------------------
loss1.899629 [   64/60000]
loss1.870006 [ 6464/60000]
loss1.756014 [12864/60000]
loss1.782025 [19264/60000]
loss1.680409 [25664/60000]
loss1.634884 [32064/60000]
loss1.629523 [38464/60000]
loss1.563246 [44864/60000]
loss1.564880 [51264/

# Save Model

In [9]:
torch.save(model.state_dict(), "model.pth") # contains model paramters

# Load Model

In [ ]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

# Making predictions

In [13]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0].to(device), test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

AttributeError: 'int' object has no attribute 'to'